# shortest path findings algorithsm 

- BFS -> Only works on unweighted graphs

- Dijkstra's Algorithm -> works on graphs whose edge weights are nonnegative

- Bellman-Ford algorithm -> works on graphs whose edge weights may be negativ

# Dijkstra's Algorithm (weighted version of BFS)

- Time complexity:  O(E+VlogV)
- Space complexiy: O(V)
- Algorithm:
    - create a hash map to represent a graph as an adjacncy list (items = (dest, cost))
    - initalize a list to keep the path weight from source to each node with an infinte value.
    - initailze heap with the staring node (similar to BFS, in BFS use queue though).
    - iterative over the heap until it gets empty.
        - Pop the top node currNode from the priority queue.
        - Traverse all outgoing edges connected to currNode using the graph hash map.
        - Add the adjacent node neighborNode to the priority queue only if the current path takes less time than the value of the adjacent node. 
        - Update the node value to current path time.

In [2]:
def dijks(adj_list, source, n):

    dist_node = {v: float("Inf") for v in range(n)}
    
    dist_node[source] = 0

    heap = [(0, source)] # (distance_to_node, node)

    while len(heap):

        dist, u = heapq.heappop(min_heap)

        for neigh, weight in adj_list[v]:

            neigh_dist = dist + weight
            
            if neigh_dist < dist_node[neigh]:
                
                dist_node[neigh] = neigh_dist
                
                heapq.heappush(heap, neigh_dist)

    return dist_node

# Problem: Path With Minimum Effort

You are a hiker preparing for an upcoming hike. You are given heights, a 2D array of size rows x columns, where heights[row][col] represents the height of cell (row, col). You are situated in the top-left cell, (0, 0), and you hope to travel to the bottom-right cell, (rows-1, columns-1) (i.e., 0-indexed). You can move up, down, left, or right, and you wish to find a route that requires the minimum effort.

A route's effort is the maximum absolute difference in heights between two consecutive cells of the route.

Return the minimum effort required to travel from the top-left cell to the bottom-right cell.

```
Input: heights = [[1,2,2],[3,8,2],[5,3,5]]
Output: 2
Explanation: The route of [1,3,5,3,5] has a maximum absolute difference of 2 in consecutive cells.
This is better than the route of [1,2,2,2,5], where the maximum absolute difference is 3.
```

In [4]:
from typing import List
class Solution:
    def minimumEffortPath(self, heights: List[List[int]]) -> int:
        graph = {}
        n_rows = len(heights)
        n_cols = len(heights[0])
        directions = [(-1,0), (+1,0), (0, +1), (0, -1)]
        
        for x in range(n_rows):
            for y in range(n_cols):
                if (x, y) not in graph:
                    graph[(x, y)] = []
                for di in directions: 
                    nei_x = x + di[0]
                    nei_y = y + di[1]
                    if (nei_x >=0 and nei_x<n_rows and nei_y>=0 and nei_y<n_cols):
                        graph[(x,y)].append((nei_x, nei_y, abs(heights[x][y]-heights[nei_x][nei_y])))

        # goal: finding the minimum cost to move from (0,0) to (n_rows-1, n_cols-1)
        # i.e., finding the shortest path between two nodes in a weighted graph, undirected
        # shortest path means BFS if the graph is unweighted. 
        # if the graph is weighted with non-negative weights, we apply Dijkestra's algorithm
        # (source_x, source_y, dist)
        heap = [(0,0,0)]
        dist = {(i,j):float('inf') for i in range(n_rows) for j in range(n_cols)}
        dist[(0,0)] = 0
        
        while len(heap)>0:
            node_x, node_y, node_dist = heapq.heappop(heap)

            for nei_x, nei_y, nei_weight in graph[(node_x, node_y)]:
                nei_dist = max([node_dist, nei_weight])
                if nei_dist < dist[(nei_x, nei_y)]:
                    dist[(nei_x, nei_y)] = nei_dist
                    heapq.heappush(heap, (nei_x, nei_y, nei_dist))
        return dist[(n_rows-1, n_cols-1)]

# Problem: Cheapest Flights Within K Stops

There are n cities connected by some number of flights. You are given an array flights where flights[i] = [fromi, toi, pricei] indicates that there is a flight from city fromi to city toi with cost pricei.

You are also given three integers src, dst, and k, return the cheapest price from src to dst with at most k stops. If there is no such route, return -1.


```
Input: n = 4, flights = [[0,1,100],[1,2,100],[2,0,100],[1,3,600],[2,3,200]], src = 0, dst = 3, k = 1
Output: 700
Explanation:
The graph is shown above.
The optimal path with at most 1 stop from city 0 to 3 is marked in red and has cost 100 + 600 = 700.
Note that the path through cities [0,1,2,3] is cheaper but is invalid because it uses 2 stops.
```

In [5]:
class Solution:
    def findCheapestPrice(self, n: int, flights: List[List[int]], src: int, dst: int, k: int) -> int:
        graph = {}
        for [u,v,w] in flights:
            if u not in graph:
                graph[u] = []
            if v not in graph:
                graph[v] = []
            graph[u].append((v,w))
        
        # Dijkstra
        dist_node = {i: float('inf') for i in range(n)}
        current_stops = {i: float('inf') for i in range(n)}
        dist_node[src] = 0 
        current_stops[src] = 0
        
         # Data is (cost, stops, node)
        heap = [(0, 0, src)]
        
        while len(heap):
            dist_u, stops, u = heapq.heappop(heap)
            
            # if destination is reached, return the cost to get here
            if u == dst:
                return dist_u
            
            # If there are no more steps left, continue 
            if stops == k + 1:
                continue
            
            for v, weight in graph[u]:
                dist_v = dist_u + weight
                if dist_v < dist_node[v]:
                    dist_node[v] = dist_v
                    heapq.heappush(heap, (dist_v, stops+1, v))
                    current_stops[v] = stops
                elif stops < current_stops[v]:
                    heapq.heappush(heap, (dist_v, stops+1, v))
                    
        if dist_node[dst] == float('inf'):
            return -1
        return dist_node[dst]
            